In [0]:
from pyspark.sql.functions import col
from pyspark.sql.functions import when #Set Filters



seq=[(1, "Mike", 28, "Male"), (2, "Lily", 30, "Female"), (3, "Raymond", 26, "Male"), (5, "Dave", 36, "Male")]
employees=spark.createDataFrame(seq,['id','name','age','gender'])

seq2=[(1, 26000), (2, 30000), (4, 25000), (3, 20000)]
salaries=spark.createDataFrame(seq2,['id','salary'])

# Operators

## Investigation Operators

In [0]:
employees.schema

Out[15]: StructType([StructField('id', LongType(), True), StructField('name', StringType(), True), StructField('age', LongType(), True), StructField('gender', StringType(), True)])

In [0]:
employees.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- gender: string (nullable = true)



In [0]:
employees.describe('gender').display()

summary,gender
count,4
mean,null
stddev,null
min,Female
max,Male


In [0]:
employees.explain()

== Physical Plan ==
*(1) Scan ExistingRDD[id#2L,name#3,age#4L,gender#5]




## Clean

In [0]:
rev_employees = employees.withColumn('age2',col('age'))
rev_employees = rev_employees.withColumn('age3',col('age'))
rev_employees = rev_employees.drop(col('age3'))

rev_employees.display()

id,name,age,gender,age2
1,Mike,28,Male,28
2,Lily,30,Female,30
3,Raymond,26,Male,26
5,Dave,36,Male,36


In [0]:
rev_employees = rev_employees.dropDuplicates(['gender'])
rev_employees.display()

id,name,age,gender,age2
2,Lily,30,Female,30
1,Mike,28,Male,28


In [0]:
# Modify the 'age2' column based on the 'id' condition
rev_employees = rev_employees.withColumn(
    "age2", when(rev_employees["id"] == 1,None).otherwise(rev_employees["age2"])
)

# Show the updated DataFrame
rev_employees.display()

id,name,age,gender,age2
2,Lily,30,Female,30
1,Mike,28,Male,null


In [0]:
rev_employees.fillna({"age2": 0}).display()

id,name,age,gender,age2
2,Lily,30,Female,30
1,Mike,28,Male,0


In [0]:
employees.select('gender').distinct().display()

gender
Male
Female


## Transformation

In [0]:
employees.select(col('gender')).display()

gender
Male
Female
Male
Male


In [0]:
employees.selectExpr('id','name','concat(id,name) as key').display()

id,name,key
1,Mike,1Mike
2,Lily,2Lily
3,Raymond,3Raymond
5,Dave,5Dave


# Join Types

In [0]:
employees.display()

id,name,age,gender
1,Mike,28,Male
2,Lily,30,Female
3,Raymond,26,Male
5,Dave,36,Male


In [0]:
salaries.display()

id,salary
1,26000
2,30000
4,25000
3,20000


In [0]:
Join_DF = employees.join(salaries,'id','inner')
Join_DF.display()

id,name,age,gender,salary
1,Mike,28,Male,26000
2,Lily,30,Female,30000
3,Raymond,26,Male,20000


In [0]:
Join_DF = employees.join(salaries,'id','left')
Join_DF.display()

id,name,age,gender,salary
1,Mike,28,Male,26000
2,Lily,30,Female,30000
3,Raymond,26,Male,20000
5,Dave,36,Male,null


In [0]:
Join_DF = employees.join(salaries,'id','right')
Join_DF.display()

id,name,age,gender,salary
1,Mike,28,Male,26000
2,Lily,30,Female,30000
4,null,null,null,25000
3,Raymond,26,Male,20000


In [0]:
Join_DF = employees.join(salaries,'id','outer')
Join_DF.display()

id,name,age,gender,salary
1,Mike,28,Male,26000
2,Lily,30,Female,30000
3,Raymond,26,Male,20000
4,null,null,null,25000
5,Dave,36,Male,null


In [0]:
Join_DF = employees.join(salaries,'id','leftsemi') # only get the columns from the master tables after inner join
Join_DF.display()

id,name,age,gender
1,Mike,28,Male
2,Lily,30,Female
3,Raymond,26,Male


In [0]:
Join_DF = employees.join(salaries,'id','leftanti') 
Join_DF.display()

id,name,age,gender
5,Dave,36,Male
